In [15]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

# Rading the Data

In [16]:
path_train = './data/V1.4_Training.csv'
path_test = './data/SubtaskA_EvaluationData.csv'

data_train = pd.read_csv(path_train, names = ['Id', 'Text', 'label'])
data_test = pd.read_csv(path_test, names = ['Id', 'Text', 'label'])

In [17]:
data_train.head()

,Id,Text,label
0,663_3,"""Please enable removing language code from the...",1
1,663_4,"""Note: in your .csproj file, there is a Suppor...",0
2,664_1,"""Wich means the new version not fully replaced...",0
3,664_2,"""Some of my users will still receive the old x...",0
4,664_3,"""The store randomly gives the old xap or the n...",0


In [18]:
data_test.head()

,Id,Text,label
0,9566,This would enable live traffic aware apps.,X
1,9569,Please try other formatting like bold italics ...,X
2,9576,Since computers were invented to save time I s...,X
3,9577,Allow rearranging if the user wants to change ...,X
4,9579,Add SIMD instructions for better use of ARM NE...,X


# Featurising text

* Bag of word
* Tf-Idf vector
* Avearge word2vec
* Tf-Idf weighted word2vec
* Response coding